In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [44]:
# Hyperparameters
input_size = 784  # 28x28 pixels
hidden_size = 128
output_size = 10
num_epochs = 40
batch_size = 100
learning_rate = 0.001

In [45]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transforms.ToTensor())

In [46]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [47]:
# Neural network
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        # out = self.softmax(out) Don't add softmax if using nn.CrossEntropyLoss since it already combines softmax with Negative likelyhood
        return out

In [48]:
model = MLP(input_size, hidden_size, output_size)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [49]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28*28)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/40], Step [100/600], Loss: 2.2692
Epoch [1/40], Step [200/600], Loss: 2.2581
Epoch [1/40], Step [300/600], Loss: 2.2458
Epoch [1/40], Step [400/600], Loss: 2.2316
Epoch [1/40], Step [500/600], Loss: 2.2066
Epoch [1/40], Step [600/600], Loss: 2.1855
Epoch [2/40], Step [100/600], Loss: 2.1871
Epoch [2/40], Step [200/600], Loss: 2.1533
Epoch [2/40], Step [300/600], Loss: 2.1291
Epoch [2/40], Step [400/600], Loss: 2.1065
Epoch [2/40], Step [500/600], Loss: 2.1096
Epoch [2/40], Step [600/600], Loss: 2.0900
Epoch [3/40], Step [100/600], Loss: 2.0688
Epoch [3/40], Step [200/600], Loss: 2.0205
Epoch [3/40], Step [300/600], Loss: 2.0046
Epoch [3/40], Step [400/600], Loss: 1.9874
Epoch [3/40], Step [500/600], Loss: 1.9042
Epoch [3/40], Step [600/600], Loss: 1.8856
Epoch [4/40], Step [100/600], Loss: 1.8156
Epoch [4/40], Step [200/600], Loss: 1.8247
Epoch [4/40], Step [300/600], Loss: 1.8631
Epoch [4/40], Step [400/600], Loss: 1.7436
Epoch [4/40], Step [500/600], Loss: 1.7677
Epoch [4/40

In [50]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 89.98 %
